# Lecon 1 : Deviner la suite

> **Bienvenue !** Ce notebook est interactif : tu vas lire,
> executer du code, et ecrire tes propres lignes de Python.

### Comment ca marche ?

1. Clique sur une cellule grise (c'est du code Python)
2. Appuie sur **Shift + Entree** pour l'executer
3. Le resultat s'affiche juste en dessous
4. Passe a la cellule suivante et recommence !

**Regle d'or** : execute les cellules **dans l'ordre**, de haut en bas.
Si tu sautes une cellule, la suivante risque de ne pas marcher.

---

In [ ]:
# ============================================================
# Cellule d'initialisation \u2014 execute sans lire (Shift+Entree)
# ============================================================

import json
import uuid

from IPython.display import HTML, display

_exercices_faits = set()
_NB_TOTAL = 4


def verifier(num_exercice, condition, message_ok, message_aide=""):
    """Valide un exercice avec feedback HTML vert/rouge + compteur."""
    try:
        _result = bool(condition)
    except Exception:
        _result = False
    if _result:
        _exercices_faits.add(num_exercice)
        n = len(_exercices_faits)
        barre = "\U0001f7e9" * n + "\u2b1c" * (_NB_TOTAL - n)
        display(
            HTML(
                f'<div style="padding:10px;background:#d4edda;border-left:5px solid #28a745;'
                f'margin:8px 0;border-radius:4px;font-family:system-ui,-apple-system,sans-serif">'
                f"\u2705 <b>{message_ok}</b><br>"
                f'<span style="color:#555">Progression : {barre} {n}/{_NB_TOTAL}</span></div>'
            )
        )
        if n == _NB_TOTAL:
            display(
                HTML(
                    '<div style="padding:12px;background:linear-gradient(135deg,#3949ab,#6a1b9a);'
                    "color:white;border-radius:8px;text-align:center;font-family:system-ui,-apple-system,sans-serif;"
                    'font-size:1.2em;margin:8px 0">\U0001f3c6 <b>Bravo ! Toutes les activites de cette lecon sont terminees !</b></div>'
                )
            )
    else:
        display(
            HTML(
                f'<div style="padding:10px;background:#fff3cd;border-left:5px solid #ffc107;'
                f'margin:8px 0;border-radius:4px;font-family:system-ui,-apple-system,sans-serif">'
                f"\U0001f4a1 <b>{message_aide}</b></div>"
            )
        )


def exercice(numero, titre, consigne, observation=""):
    """Affiche la banniere d'exercice."""

    def _style_code(text):
        return text.replace(
            "<code>",
            '<code style="font-size:0.95em;background:#bbdefb;'
            'padding:1px 5px;border-radius:3px;font-family:monospace;">',
        )

    obs = ""
    if observation:
        obs = (
            f'<div style="margin-top:6px;color:#555;font-size:0.92em;">'
            f"<b>Ce que tu vas voir\u00a0:</b> {_style_code(observation)}</div>"
        )
    display(
        HTML(
            f'<div style="border-left:5px solid #1565c0;background:#e8f0fe;'
            f"padding:12px 16px; margin:4px 0 10px 0; border-radius:0 8px 8px 0;"
            f'font-family:system-ui,-apple-system,sans-serif; font-size:0.95em;">'
            f'<b style="color:#0d47a1;">Exercice\u00a0{numero} \u2014 {titre}</b><br>'
            f"{_style_code(consigne)}{obs}</div>"
        )
    )


def afficher_heatmap(compteur, titre="Heatmap des bigrammes"):
    """Heatmap interactive des bigrammes (survol = detail, highlight ligne/colonne)."""
    uid = uuid.uuid4().hex[:8]
    lettres = sorted(
        set(k for k in compteur) | set(s for v in compteur.values() for s in v)
    )
    max_count = max((c for v in compteur.values() for c in v.values()), default=1)
    # En-tete du tableau
    header = "".join(
        f'<th style="padding:2px 4px;font-size:0.75em">{ch}</th>' for ch in lettres
    )
    # Lignes de donnees avec data-attributes pour le JS
    rows = ""
    for i, l1 in enumerate(lettres):
        cells = ""
        for j, l2 in enumerate(lettres):
            count = compteur.get(l1, {}).get(l2, 0)
            opacity = count / max_count if max_count > 0 else 0
            bg = f"rgba(21,101,192,{opacity:.2f})"
            cells += (
                f'<td data-r="{i}" data-c="{j}" data-v="{count}"'
                f' style="padding:2px 4px;background:{bg};text-align:center;'
                f"font-size:0.65em;border:1px solid #eee;min-width:16px;"
                f'cursor:crosshair">{count if count > 0 else ""}</td>'
            )
        rows += (
            f'<tr><th style="padding:2px 4px;font-size:0.75em">{l1}</th>{cells}</tr>'
        )
    lettres_js = json.dumps(lettres)
    hint = "Survole une case pour voir le detail"
    # HTML + JS interactif (hover = highlight ligne/colonne + info)
    display(
        HTML(
            f"<!-- tuto-viz -->"
            f'<div style="margin:8px 0;overflow-x:auto"><b>{titre}</b>'
            f'<table id="t{uid}" style="border-collapse:collapse;margin-top:4px">'
            f"<tr><th></th>{header}</tr>{rows}</table>"
            f'<div id="i{uid}" style="margin-top:4px;color:#555;font-size:0.85em;'
            f'min-height:1.3em">{hint}</div></div>'
            f'<script>(function(){{"use strict";'
            f'var t=document.getElementById("t{uid}"),'
            f'info=document.getElementById("i{uid}"),'
            f"L={lettres_js};"
            f'function hl(r,c){{t.querySelectorAll("td[data-r]").forEach(function(d){{'
            f"var a=+d.dataset.r===r||+d.dataset.c===c;"
            f'd.style.outline=a?"2px solid #1565c0":"";'
            f'd.style.position=a?"relative":""}});}};'
            f't.addEventListener("mouseover",function(e){{'
            f'var d=e.target.closest("td[data-r]");if(!d)return;'
            f"var r=+d.dataset.r,c=+d.dataset.c;hl(r,c);"
            f'info.textContent=L[r]+" \u2192 "+L[c]+" : "+d.dataset.v+" fois"}});'
            f't.addEventListener("mouseleave",function(){{hl(-1,-1);'
            f'info.textContent="{hint}"}})'
            f"}})();</script>"
        )
    )


def afficher_barres(valeurs, etiquettes, titre="Probabilites", couleur="#1565c0"):
    """Barres de probabilites animees (apparition progressive + hover)."""
    max_val = max(valeurs) if valeurs else 1
    n = len(valeurs)
    bar_h, gap, lbl_w, bar_w = 24, 4, 30, 220
    svg_h = n * (bar_h + gap) + gap
    svg_w = lbl_w + bar_w + 70
    bars = ""
    for i, (etiq, val) in enumerate(zip(etiquettes, valeurs, strict=False)):
        pct = val / max_val if max_val > 0 else 0
        w = max(pct * bar_w, 2)
        y = i * (bar_h + gap) + gap
        delay = f"{i * 0.08:.2f}"
        bars += (
            f'<text x="{lbl_w - 4}" y="{y + bar_h * 0.72}" '
            f'text-anchor="end" font-size="13" font-weight="bold" '
            f'fill="#333">{etiq}</text>'
            f'<rect x="{lbl_w}" y="{y}" width="0" height="{bar_h}" '
            f'rx="4" fill="{couleur}" opacity="0.85">'
            f'<animate attributeName="width" from="0" to="{w:.0f}" '
            f'dur="0.5s" begin="{delay}s" fill="freeze"/></rect>'
            f'<text x="{lbl_w + bar_w + 8}" y="{y + bar_h * 0.72}" '
            f'font-size="12" fill="#555">{val:.0%}</text>'
        )
    display(
        HTML(
            f"<!-- tuto-viz -->"
            f'<div style="margin:8px 0"><b>{titre}</b>'
            f'<svg width="{svg_w}" height="{svg_h}" '
            f'style="margin-top:4px;display:block">{bars}</svg></div>'
        )
    )


def afficher_generation(mot, delai_ms=300):
    """Anime l'apparition d'un mot lettre par lettre."""
    uid = uuid.uuid4().hex[:8]
    lettres_js = json.dumps(list(mot))
    display(
        HTML(
            f"<!-- tuto-viz -->"
            f'<div style="margin:8px 0;font-family:monospace;font-size:1.3em">'
            f'<span id="g{uid}" style="letter-spacing:2px"></span>'
            f'<span id="c{uid}" style="animation:bk{uid} 0.7s infinite">|</span></div>'
            f"<style>@keyframes bk{uid}{{0%,100%{{opacity:1}}50%{{opacity:0}}}}</style>"
            f"<script>(function(){{"
            f"var L={lettres_js},i=0,"
            f'el=document.getElementById("g{uid}"),'
            f'cur=document.getElementById("c{uid}");'
            f"var iv=setInterval(function(){{"
            f'if(i>=L.length){{clearInterval(iv);cur.style.display="none";return}}'
            f'var s=document.createElement("span");'
            f"s.textContent=L[i];"
            f's.style.cssText="opacity:0;transition:opacity 0.3s";'
            f"el.appendChild(s);"
            f'setTimeout(function(){{s.style.opacity="1"}},30);'
            f"i++}},{delai_ms})"
            f"}})();</script>"
        )
    )


print("Outils de visualisation charges !")

## Comment une IA "devine" le mot suivant ?

Quand tu ecris un message sur ton telephone, il te **propose** le mot suivant.
Comment fait-il ? Il a appris quels mots viennent souvent apres d'autres.

On va faire pareil, mais avec des **lettres** au lieu de mots.

Par exemple, pense a ces suites :
- **A, B, C** \u2192 quelle est la suite ? **D** (l'alphabet !)
- **L, U, N, D** \u2192 quelle est la suite ? **I** (les jours de la semaine)
- **Q** \u2192 quelle lettre vient presque toujours apres ? ...

In [ ]:
exercice(
    1,
    "Quelle lettre suit Q ?",
    "Change <code>ma_reponse</code> ci-dessous, puis <b>Shift + Entree</b>. Quelle lettre vient presque toujours apres Q ?",
    "Si ta reponse est bonne, la box passe au vert.",
)

# --- Code de preparation (ne touche pas) ---
print("Reponses :")
print("  A, B, C -> D  (l'alphabet !)")
print("  L, U, N, D -> I  (les jours : Lundi)")

# ╔══════════════════════════════════════╗
# ║  A TOI DE JOUER !                    ║
# ╠══════════════════════════════════════╣

ma_reponse = "?"  # <-- Quelle lettre vient apres Q ?

# ╚══════════════════════════════════════╝

print(f"  Q -> {ma_reponse.upper()}  (ta reponse !)")

# Validation exercice 1
verifier(
    1,
    ma_reponse.lower() == "u",
    "Bravo ! Apres Q vient presque toujours U.",
    "Pas tout a fait... quelle lettre vient presque toujours apres Q ?",
)

---
## Etape 1 : Charger nos Pokemon

On va prendre une liste de noms de Pokemon et compter quelle lettre
vient apres quelle autre. Ces 20 noms sont notre **dataset**
(les donnees d'entrainement).

Execute la cellule ci-dessous pour charger nos 20 Pokemon :

In [ ]:
# Nos 20 Pokemon : le "dataset" de cette lecon
pokemons = [
    "arcanin",  # 7 lettres
    "bulbizarre",  # 10 lettres
    "carapuce",  # 8 lettres
    "dracaufeu",  # 9 lettres
    "ectoplasma",  # 10 lettres
    "evoli",  # 5 lettres
    "felinferno",  # 10 lettres
    "gardevoir",  # 9 lettres
    "goupix",  # 6 lettres
    "lokhlass",  # 8 lettres
    "lucario",  # 7 lettres
    "metamorph",  # 9 lettres
    "mewtwo",  # 6 lettres
    "noctali",  # 7 lettres
    "pikachu",  # 7 lettres
    "rondoudou",  # 9 lettres
    "ronflex",  # 7 lettres
    "salameche",  # 9 lettres
    "togepi",  # 6 lettres
    "voltali",  # 7 lettres
]

print(f"On a {len(pokemons)} Pokemon pour apprendre.")
print()
print("Les 5 premiers :", pokemons[:5])

Explore un peu cette liste avant de continuer :

In [ ]:
exercice(
    2,
    "Explore la liste de Pokemon",
    'Complete les variables ci-dessous avec <code>pokemons[-1]</code> et <code>len("bulbizarre")</code>.',
    "Tu vas voir le dernier Pokemon et le nombre de lettres.",
)

# --- Code de preparation (ne touche pas) ---
# pokemons[-1] donne le dernier element d'une liste
# len("mot") donne le nombre de lettres

# ╔══════════════════════════════════════╗
# ║  A TOI DE JOUER !                    ║
# ╠══════════════════════════════════════╣

dernier_pokemon = ...  # <-- Ecris pokemons[-1]
nb_lettres = ...  # <-- Ecris len("bulbizarre")

# ╚══════════════════════════════════════╝

print(f"Le dernier Pokemon est : {dernier_pokemon}")
print(f"'bulbizarre' a {nb_lettres} lettres")

# Validation exercice 2
verifier(
    2,
    isinstance(dernier_pokemon, str)
    and isinstance(nb_lettres, int)
    and dernier_pokemon == pokemons[-1]
    and nb_lettres == len("bulbizarre"),
    f"Bravo ! {dernier_pokemon} est le dernier, et 'bulbizarre' a {nb_lettres} lettres.",
    "Remplace les ... par pokemons[-1] et len('bulbizarre').",
)

---
## Etape 2 : Compter les paires de lettres (bigrammes)

Maintenant, on va compter **quelle lettre vient apres quelle autre**.
Par exemple, dans "pikachu" :
- apres "p" vient "i"
- apres "i" vient "k"
- apres "k" vient "a"
- ...

On utilise un **point** `.` pour marquer le debut et la fin :
`.pikachu.` signifie "le nom commence et finit ici".

Ces paires de lettres s'appellent des **bigrammes** :

In [ ]:
from collections import Counter

# Comptons : apres chaque lettre, quelle lettre vient ensuite ?
compteur = {}

for pokemon in pokemons:
    mot = "." + pokemon + "."  # ex: ".pikachu."
    for i in range(len(mot) - 1):
        lettre_actuelle = mot[i]  # la lettre courante
        lettre_suivante = mot[i + 1]  # celle juste apres
        if lettre_actuelle not in compteur:
            compteur[lettre_actuelle] = Counter()
        compteur[lettre_actuelle][lettre_suivante] += 1

# Que vient-il apres la lettre 'a' ?
print("Apres la lettre 'a', on trouve :")
for lettre, nb in compteur["a"].most_common():
    print(f"  '{lettre}' -> {nb} fois")

print()
print("Visualisation interactive (survole les cases) :")

# Heatmap interactive : survole pour voir le detail
afficher_heatmap(compteur, titre="Heatmap : quelle lettre suit quelle autre ?")

**Qu'est-ce que tu remarques ?**

- Quelles paires de lettres sont les plus frequentes (cases foncees) ?
- Est-ce que certaines lettres ne se suivent jamais (cases vides) ?
- Regarde la ligne du `.` (debut de mot) : quelles lettres commencent les noms ?

---

In [ ]:
exercice(
    3,
    "Explore les paires de lettres",
    'Change <code>ma_lettre</code> ci-dessous (essaie <code>"p"</code>, <code>"."</code> ou ta lettre preferee).',
    "Les comptages changent selon la lettre choisie.",
)

# ╔══════════════════════════════════════╗
# ║  A TOI DE JOUER !                    ║
# ╠══════════════════════════════════════╣

ma_lettre = "a"  # <-- Change cette lettre !

# ╚══════════════════════════════════════╝

print(f"Apres la lettre '{ma_lettre}', on trouve :")
for lettre, nb in compteur[ma_lettre].most_common():
    print(f"  '{lettre}' -> {nb} fois")

# Validation exercice 3
verifier(
    3,
    ma_lettre != "a",
    f"Bien joue ! Tu as explore les suites de '{ma_lettre}'.",
    "Change ma_lettre pour une autre lettre, par exemple 'p' ou '.'.",
)

---
## Etape 3 : Transformer les comptes en probabilites

Au lieu de dire "la lettre 'r' vient 3 fois apres 'a'",
on veut dire "il y a 25% de chances que 'r' vienne apres 'a'".

C'est ce qu'on appelle une **probabilite** : un nombre entre 0% et 100%
qui dit "a quel point c'est probable".

On divise chaque compte par le total :

In [ ]:
# Transformer les comptes en probabilites
probas = {}

for lettre, suivantes in compteur.items():
    total = sum(suivantes.values())  # nombre total de suites observees
    probas[lettre] = {}
    for suivante, nb in suivantes.items():
        probas[lettre][suivante] = nb / total  # compte / total = probabilite

# Affichons les probabilites apres 'a'
print("Probabilites apres 'a' :")
for lettre, p in sorted(probas["a"].items(), key=lambda x: -x[1]):
    barre = "#" * int(p * 40)  # barre proportionnelle
    print(f"  '{lettre}' : {p:.0%} {barre}")

# Barres animees : les 8 lettres les plus probables apres 'a'
_top = sorted(probas["a"].items(), key=lambda x: -x[1])[:8]
afficher_barres(
    [v for _, v in _top], [k for k, _ in _top], titre="Probabilites apres 'a'"
)

---
## Etape 4 : Generer un nom de Pokemon !

Maintenant on peut **inventer** un nom de Pokemon :
1. On part du debut (le point `.`)
2. On choisit la lettre suivante au hasard, en respectant les probabilites
3. On recommence jusqu'a tomber sur un point `.` (fin du nom)

C'est exactement comme ca que ChatGPT fonctionne, mais avec des **mots**
au lieu de lettres, et des milliards de parametres au lieu de 20 Pokemon.

In [ ]:
import random


def generer_pokemon(probas):
    """Genere un nom de Pokemon lettre par lettre."""
    nom = ""
    lettre = "."  # on commence au debut

    while True:
        # Choisir la lettre suivante selon les probabilites
        choix = list(probas[lettre].keys())  # lettres possibles
        poids = list(probas[lettre].values())  # leurs probabilites
        lettre = random.choices(choix, weights=poids, k=1)[0]

        if lettre == ".":  # fin du nom
            break
        nom += lettre

    return nom


# Generons 10 noms de Pokemon !
print("Noms de Pokemon inventes par notre modele :")
print()
noms_generes = []
for i in range(10):
    nom = generer_pokemon(probas)
    noms_generes.append(nom)
    print(f"  {i + 1}. {nom.capitalize()}")

# Animation du dernier nom genere
print()
print("Animation du dernier nom genere :")
afficher_generation(noms_generes[-1].capitalize(), delai_ms=300)

**Qu'est-ce que tu remarques ?**

- Les noms generes ressemblent-ils a des Pokemon ?
- Certains sont tres courts (1-2 lettres). Pourquoi ?
- Le modele ne regarde que **1 lettre en arriere**. Il ne sait pas
  que "Pik" est un bon debut de Pokemon.

Re-execute la cellule au-dessus pour generer d'autres noms !

---

In [ ]:
exercice(
    4,
    "Genere plein de Pokemon !",
    "Change <code>nombre</code> ci-dessous (essaie 50 ou 100).",
    "Plus tu en generes, plus tu verras de noms droles.",
)

# ╔══════════════════════════════════════╗
# ║  A TOI DE JOUER !                    ║
# ╠══════════════════════════════════════╣

nombre = 10  # <-- Mets 50 ici !

# ╚══════════════════════════════════════╝

print(f"Generation de {nombre} Pokemon :")
print()
for i in range(nombre):
    print(f"  {i + 1}. {generer_pokemon(probas).capitalize()}")

# Validation exercice 4
verifier(
    4,
    nombre != 10,
    f"Genial ! Tu as genere {nombre} Pokemon.",
    "Change nombre pour une autre valeur, par exemple 50.",
)

**Les noms ressemblent-ils a des Pokemon ? Pourquoi ?**

Notre modele a appris les **paires de lettres** frequentes dans les noms
de Pokemon. Alors les noms generes "sonnent" un peu comme du Pokemon...
mais pas toujours, parce qu'il ne regarde qu'**une seule lettre en arriere**.

---

## Ce qu'on a appris

- Un modele de langage **predit la suite** en se basant sur ce qu'il a vu avant
- Il utilise des **probabilites** : certaines lettres sont plus probables que d'autres
- Meme un modele tres simple peut generer des noms qui "sonnent" bien

### Le probleme de notre modele

Notre modele ne regarde que **1 lettre en arriere**. Il ne sait pas que
"Pik" est un bon debut de Pokemon, parce qu'il ne voit que la derniere lettre.

Dans la prochaine lecon, on va lui apprendre a **s'ameliorer quand il se trompe**.

---
*Prochaine lecon : [02 - Apprendre de ses erreurs](02_apprendre_des_erreurs.ipynb)*

---

### Sources (ISO 42001)

- **Concept de bigrammes et modeles de langage** : [microgpt.py](https://gist.github.com/karpathy/8627fe009c40f57531cb18360106ce95) -- Andrej Karpathy
- **Approche pedagogique character-level** : [Video "Let's build GPT"](https://www.youtube.com/watch?v=kCc8FmEb1nY) -- Andrej Karpathy (2023)
- **Visualisation des reseaux de neurones** : [3Blue1Brown - Neural Networks](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi) -- Grant Sanderson
- **Dataset Pokemon** : (c) Nintendo / Creatures Inc. / GAME FREAK inc., usage educatif. Source : [PokeAPI](https://pokeapi.co/)